<a href="https://colab.research.google.com/github/ShaileeGavnekar/Movie_Recommendation_System/blob/main/movie_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from difflib import get_close_matches  # For fuzzy matching

# Load the datasets
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

# Merge the datasets on the 'title' column
movies = movies.merge(credits, on='title')

# Select necessary columns
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
movies.dropna(inplace=True)

# Define helper functions
def convert(text):
    L = []
    try:
        for i in ast.literal_eval(text):
            L.append(i['name'])
    except (ValueError, SyntaxError):
        pass
    return L

def convert3(text):
    L = []
    try:
        for i in ast.literal_eval(text):
            if len(L) < 3:
                L.append(i['name'])
    except (ValueError, SyntaxError):
        pass
    return L

def fetch_director(text):
    L = []
    try:
        for i in ast.literal_eval(text):
            if i['job'] == 'Director':
                L.append(i['name'])
    except (ValueError, SyntaxError):
        pass
    return L

def collapse(L):
    return [i.replace(" ", "") for i in L]

# Apply the functions to the columns
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies['cast'] = movies['cast'].apply(convert3)
movies['crew'] = movies['crew'].apply(fetch_director)

movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

movies['overview'] = movies['overview'].apply(lambda x: x.split())
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

new = movies[['movie_id', 'title', 'tags']]
new['tags'] = new['tags'].apply(lambda x: " ".join(x))

# Normalize titles to lowercase and strip spaces
new['title'] = new['title'].str.lower().str.replace(" ", "")

cv = CountVectorizer(max_features=5000, stop_words='english')
vector = cv.fit_transform(new['tags']).toarray()

similarity = cosine_similarity(vector)




<ipython-input-17-5991dddc91b0>:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['tags'] = new['tags'].apply(lambda x: " ".join(x))
<ipython-input-17-5991dddc91b0>:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['title'] = new['title'].str.lower().str.replace(" ", "")


In [ ]:
def recommend(movie):
    movie_normalized = movie.lower().replace(" ", "")  # Normalize input
    closest_matches = get_close_matches(movie_normalized, new['title'].values, n=1, cutoff=0.6)  # Fuzzy matching

    if not closest_matches:
        print(f"Movie '{movie}' not found in dataset.")
        return

    matched_title = closest_matches[0]
    index = new[new['title'] == matched_title].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])

    print(f"Recommendations for '{movie}':")
    for i in distances[1:6]:
        print(new.iloc[i[0]].title.title())

# Example usage
recommend('Iron Man')

Recommendations for 'Iron Man':
Ironman2
Ironman3
Avengers:Ageofultron
Captainamerica:Civilwar
Theavengers
